In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [3]:
def txt_to_dataframe(file_path):
    with open(file_path, 'r') as file:
        # Read lines, remove newline characters, and filter out empty lines
        lines = [line.strip() for line in file if line.strip()]

    # Convert list of lines to a DataFrame
    df = pd.DataFrame(lines, columns=['Text'])

    return df

In [4]:
def write_google_sheet(df, sheet_name):
    sh = gc.create(sheet_name)
    worksheet = gc.open(sheet_name).sheet1
    worksheet.append_rows([df.columns.values.tolist()] + df.values.tolist())
    # worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    print(worksheet.get_all_records())

In [5]:
import os

def process_folders_and_files(root_dir):
    # Walk through all directories and files in the root directory
    for subdir, dirs, files in os.walk(root_dir):
        for filename in files:
            # Check if the file ends with .txt
            if filename.endswith('.txt'):
                file_path = os.path.join(subdir, filename)

                # Call the txt_to_dataframe function
                df = txt_to_dataframe(file_path)
                sections = file_path.split('/')
                last_three_sections = sections[-3:]
                resulting_path = '/'.join(last_three_sections)
                print(resulting_path)
                write_google_sheet(df, resulting_path)

                print(f"Processed {file_path}, DataFrame shape: {df.shape}")


In [8]:
root_dir = '/content/drive/MyDrive/Lung Ventilator LLM/LUNG LLM/5/With Image Description/Results - GPT4 with Images/Issues'  # Replace with the path to your directory
process_folders_and_files(root_dir)

Results - GPT4 with Images/Issues/global.txt
[{'Text': "Based on the detailed Software Requirements Specification provided for the Mechanical Lung Ventilator, let's address the issues raised:"}, {'Text': '### Issue #2: Setting Ventilation Parameters with Patient Connected'}, {'Text': "According to FUN10.5, it is indeed stated that in Standby Mode, ventilation is off, and it is possible to adjust all user-controlled parameters for ventilation and alarms before connecting to the patient and starting patient ventilation. However, the GUI and Controller state machines (Figures 3.1 and 4.1) and their descriptions do not explicitly forbid parameter adjustments with the patient connected, especially in Settings Mode (GUI.7, GUI.8, CONT.8, CONT.10). The system transitions to Settings Mode to adjust parameters, which can occur during ventilation (GUI.108, CONT.9, CONT.10). Therefore, while FUN10.5 suggests parameter adjustments should be made before patient connection, the system's design accom